#### 1. W2V 학습

In [82]:
import nltk
import time
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from gensim.models import KeyedVectors
from sklearn.svm import SVC
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
import string
from sklearn.decomposition import PCA
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from sklearn.metrics import mean_squared_error, f1_score, roc_auc_score


#################################3
from gensim.models import KeyedVectors
import gensim.downloader as api

# 구글의 Word2Vec 모델 다운로드
# w2v_model = api.load("word2vec-google-news-300")
# print(f"{'Downloaded':=^30}")

# # 모델을 로컬에 저장
# w2v_model.save('/home/hooni/gensim-data/word2vec-google-news-300/word2vec-google-news-300')
# print(f"{'Saved':=^30}")
#####################################


# Word2Vec 모델 생성
vector_size = 300

# Pre-trained 모델
# model = w2v_model
w2v_model_path = '/home/hooni/gensim-data/word2vec-google-news-300/word2vec-google-news-300'
model = KeyedVectors.load(w2v_model_path)
print(f"{'Loaded':=^30}")

# 완료 표시
print('Job done')

============Loaded============
Job done


#### 2. 워드 임베딩

In [44]:
total_file_path = "data/dataset.txt"
total_documents = []
total_document = []

stop_words = set(stopwords.words('english'))

with open(total_file_path, 'r', encoding='utf-8') as file:
    for line in file:
        # Document separation
        if line == '\n':
            total_documents.append(total_document)
            total_document = []
            
        # Tokenize
        tokens = line.lower().strip().split()  # 문장을 공백으로 나누어 단어 리스트로 변환
        
        # Preprocess
        tokens = [word for word in tokens if word.isalpha()]  # 문자열만 포함
        tokens = [word for word in tokens if word not in stop_words]  # 불용어 제거
        tokens = [word for word in tokens if word not in string.punctuation]  # 구두점 제거
    
        # Remove empty list
        if len(tokens) == 0:
            continue
        
        # Add to document
        total_document.extend(tokens)

    # For last game document
    total_documents.append(total_document)
    
    # W2V 모델 로드
# model = Word2Vec.load("word2vec.model")  # 위에서 생성한 모델 파일명을 사용해야 합니다

# 워드 임베딩
weird_document_index = []
total_document_vectors = np.array([]).reshape(0, vector_size)
for document_index, total_document in enumerate(total_documents):
    # 한 게임에 대한 임베딩
    document_vectors = np.array([]).reshape(0, vector_size)
    
    # 게임 내 한 단어에 대한 임베딩
    count = 0
    for word in total_document:
        if word in model:
            document_vectors = np.vstack((document_vectors, model[word]))
            count += 1
        # For debug
        # else:
        #     print(word)
    
    if count > 0:
        # Feature vector of a document (game)
        document_vectors = document_vectors.mean(axis=0)

        # Append to total document vector
        total_document_vectors = np.vstack((total_document_vectors, document_vectors))
    else:
        print(f'[#{document_index}] Weird paragraph: {total_document}')
        weird_document_index.append(document_index)
    
# Debug
print('Job done')

[#150] Weird paragraph: []
[#315] Weird paragraph: []
[#512] Weird paragraph: ['chindi']
Job done


#### 3. 목표값 붙이기

In [103]:
file_path = 'data/classification.txt'
class_list = []

class_size = 15
with open(file_path, 'r') as file:
    for line in file:
        class_type = str(line.strip())
        class_list.append(class_type)
class_list = np.array(class_list)

# Remove weird document
class_list = np.delete(class_list, weird_document_index)

# Encoding
ohe = OneHotEncoder(sparse=False)
le = LabelEncoder()
le_label = le.fit_transform(np.array(class_list)).reshape(-1, 1)
ohe_label = ohe.fit_transform(np.array(class_list).reshape(-1, 1))

label_size = le_label.shape[1] + ohe_label.shape[1]
data_index = np.arange(1, total_document_vectors.shape[0] + 1).reshape(-1, 1)
data = np.hstack((data_index, total_document_vectors, le_label, ohe_label))

#debug
print('jobs finished b')

jobs finished b


/home/hooni/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [110]:
test_ratio = 0.2

# 데이터셋 분할
test_ratio = 0.2
index_size = 1
X_train, X_test, y_train, y_test = train_test_split(data[:, :-label_size],
                                                    data[:, -label_size:],
                                                    test_size=test_ratio,
                                                    stratify=data[:, -label_size:-label_size + le_label.shape[1]],
                                                    random_state=0)

# 자료형 변환
X_train_index, X_test_index = X_train[:, :index_size].astype(int), X_test[:, :index_size].astype(int)
X_train, X_test = X_train[:, index_size:], X_test[:, index_size:]
# X_train = X_train.flatten()
# X_test = X_test.flatten()
y_train_ohe = y_train[:, le_label.shape[1]:].astype(np.float64)
y_test_ohe = y_test[:, le_label.shape[1]:].astype(np.float64)
y_train_le = y_train[:, :le_label.shape[1]].astype(np.float64)
y_test_le = y_test[:, :le_label.shape[1]].astype(np.float64)

#X_train_res = X_train.reshape((-1,1))

# Result Preparation
result_table = pd.DataFrame(columns=['index', 'type', 'model', 'truth', 'prediction', 'f1', 'auc'])

print('debug')

debug


#### 4. Model Train

##### 4-1. LR

In [121]:
# LR 학습
timer = time.time()
lr_model = LogisticRegression(multi_class='multinomial')
lr_model.fit(X_train, y_train_le) #label-encoding 형태로 받음
print(f'Elapsed time for LR training: {(time.time() - timer)/60:0.2f} min')

# LR 학습데이터 추론
print('[LR 학습 데이터]')
train_prediction = lr_model.predict(X_train)
print(f'{y_train=}')
print(f'{train_prediction=}')

# label-encoding -> one-hot-encoding (결과 도출을 위한 형태)
train_prediction_ohe = ohe.transform(le.inverse_transform(train_prediction.astype(int)).reshape(-1, 1))

f1 = f1_score(y_train_ohe, train_prediction_ohe, average='weighted')
auc = roc_auc_score(y_train_ohe, train_prediction_ohe)
for index in range(train_prediction.shape[0]):
    result_table.loc[result_table.shape[0]] = [X_train_index[index, 0], 'train', 'LR', y_train_le[index, 0], train_prediction[index],
                                               f1_score(y_train_ohe, train_prediction_ohe, average='weighted'),
                                               roc_auc_score(y_train_ohe, train_prediction_ohe)]
print(f'result: {f1=:0.3f}, {auc=:0.3f}')

# LR 추론
print('[LR 검증 데이터]')
test_prediction = lr_model.predict(X_test)
print(f'{y_test=}')
print(f'{test_prediction=}')

test_prediction_ohe = ohe.transform(le.inverse_transform(test_prediction.astype(int)).reshape(-1, 1))
f1 = f1_score(y_test_ohe, test_prediction_ohe, average='weighted')
auc = roc_auc_score(y_test_ohe, test_prediction_ohe)
for index in range(test_prediction.shape[0]):
    result_table.loc[result_table.shape[0]] = [X_test_index[index, 0], 'test', 'LR', y_test_le[index, 0], test_prediction[index],
                                               f1_score(y_test_ohe, test_prediction_ohe, average='weighted'),
                                               roc_auc_score(y_test_ohe, test_prediction_ohe)]
print(f'result: {f1=:0.3f}, {auc=:0.3f}')

/home/hooni/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Elapsed time for LR training: 0.00 min
[LR 학습 데이터]
y_train=array([[12.,  0.,  0., ...,  0.,  1.,  0.],
       [13.,  0.,  0., ...,  0.,  0.,  1.],
       [12.,  0.,  0., ...,  0.,  1.,  0.],
       ...,
       [ 3.,  0.,  0., ...,  0.,  0.,  0.],
       [ 3.,  0.,  0., ...,  0.,  0.,  0.],
       [ 8.,  0.,  0., ...,  0.,  0.,  0.]])
train_prediction=array([13., 13., 13., 13., 13., 13., 13., 13., 13., 13., 13., 13., 13.,
       13., 13., 13., 13., 13., 13., 13., 13., 13., 13.,  3., 13., 13.,
       13., 13.,  9.,  9., 13., 13.,  3., 13., 13., 13., 13., 13., 13.,
       13., 13.,  3., 13., 13.,  3., 13., 13., 13., 13., 13., 13., 13.,
       13.,  3., 13., 13.,  3., 13., 13., 13., 13., 13., 13., 13., 13.,
       13.,  3., 13., 13., 13., 13., 13., 13.,  3., 13., 13., 13., 13.,
       12., 13., 13., 13., 13., 13., 13., 13., 13., 13., 13., 13.,  9.,
       13., 13., 13., 13., 13., 13., 13., 13., 13., 13., 13., 13.,  3.,
       13., 13., 13., 13., 13., 13., 13., 13., 13., 13., 13., 13., 13.,

##### 4-2. SVM

In [122]:
param_space = {
    'C': (1, 10000),
    'gamma': (0.01, 1.0)
}

# 하이퍼 파라미터 튜닝
svc_rbf = SVC(kernel='rbf')
opt = BayesSearchCV(svc_rbf, param_space, n_iter=50, cv=10, n_jobs=-1, n_points=8, verbose=0)

timer = time.time()


opt.fit(X_train, y_train_le.ravel())
print(f'Elapsed time for SVC optimization: {(time.time() - timer)/60:0.2f} min')
print("최적의 하이퍼파라미터:", opt.best_params_)

# SVC 학습
timer = time.time()
svc_rbf = SVC(kernel='rbf', **opt.best_params_)
svc_rbf.fit(X_train, y_train_le.ravel())
print(f'Elapsed time for SVC training: {(time.time() - timer)/60:0.2f} min')

# SVM 학습데이터 추론
print('[SVM 학습 데이터]')
train_prediction = svc_rbf.predict(X_train)
# print(f'{y_train_le=}')
# print(f'{train_prediction=}')

# label-encoding -> one-hot-encoding (결과 도출을 위한 형태)
train_prediction_ohe = ohe.transform(le.inverse_transform(train_prediction.astype(int)).reshape(-1, 1))
f1 = f1_score(y_train_ohe, train_prediction_ohe, average='weighted')
auc = roc_auc_score(y_train_ohe, train_prediction_ohe) ####################one_hot_encoding 제외
for index in range(train_prediction.shape[0]):
    result_table.loc[result_table.shape[0]] = [X_train_index[index], 'train', 'SVM', y_train_le[index], train_prediction[index],
                                               f1_score(y_train_ohe, train_prediction_ohe, average='weighted'),
                                               roc_auc_score(y_train_ohe, train_prediction_ohe)]  ######one_hot_encoding 제외
print(f'result: {f1=:0.3f}, {auc=:0.3f}')

# SVM 추론
print('[SVM 검증 데이터]')
test_prediction = svc_rbf.predict(X_test)
# print(f'{y_test=}')
# print(f'{test_prediction=}')
test_prediction_ohe = ohe.transform(le.inverse_transform(test_prediction.astype(int)).reshape(-1, 1))
f1 = f1_score(y_test_ohe, test_prediction_ohe, average='weighted')
auc = roc_auc_score(y_test_ohe, test_prediction_ohe) ######one_hot_encoding 제외
for index in range(test_prediction.shape[0]):
    result_table.loc[result_table.shape[0]] = [X_test_index[index], 'test', 'SVM', y_test_le[index], test_prediction[index],
                                               f1_score(y_test_ohe, test_prediction_ohe, average='weighted'),
                                               roc_auc_score(y_test_ohe, test_prediction_ohe)] ######one_hot_encoding 제외
print(f'result: {f1=:0.3f}, {auc=:0.3f}')

/home/hooni/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  warnings.warn(
/home/hooni/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  warnings.warn(
/home/hooni/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  warnings.warn(
/home/hooni/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  warnings.warn(
/home/hooni/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  

Elapsed time for SVC optimization: 0.17 min
최적의 하이퍼파라미터: OrderedDict([('C', 205), ('gamma', 1.0)])
Elapsed time for SVC training: 0.00 min
[SVM 학습 데이터]
result: f1=1.000, auc=1.000
[SVM 검증 데이터]
result: f1=0.564, auc=0.651


##### 4-3. rf

In [123]:
param_space = {
    'n_estimators': Integer(10, 200),
    'max_depth': Integer(1, 32),
    'min_samples_split': Real(0.1, 1.0),
    'min_samples_leaf': Real(0.1, 0.5),
    'max_features': Categorical(['sqrt', 'log2'])
}

# 하이퍼 파라미터 튜닝
rf_model = RandomForestRegressor()
opt = BayesSearchCV(rf_model, param_space, n_iter=50, cv=10, n_jobs=-1, n_points=8, verbose=0)

timer = time.time()
opt.fit(X_train, y_train_le.ravel())
print(f'Elapsed time for RF optimization: {(time.time() - timer)/60:0.2f} min')
print("최적의 하이퍼파라미터:", opt.best_params_)

# RF 학습
timer = time.time()
rf_model = RandomForestRegressor(**opt.best_params_)
rf_model.fit(X_train, y_train_le.ravel())
print(f'Elapsed time for RF training: {(time.time() - timer)/60:0.2f} min')

# label-encoding -> one-hot-encoding (결과 도출을 위한 형태)
print('[RF 학습 데이터]')
train_prediction = rf_model.predict(X_train)
print(f'{y_train=}')
print(f'{train_prediction=}')

train_prediction_ohe = ohe.transform(le.inverse_transform(train_prediction.astype(int)).reshape(-1, 1))
f1 = f1_score(y_train_ohe, train_prediction_ohe, average='weighted')
auc = roc_auc_score(y_train_ohe, train_prediction_ohe)
for index in range(train_prediction.shape[0]):
    result_table.loc[result_table.shape[0]] = [X_train_index[index, 0], 'train', 'RF', y_train_le[index, 0], train_prediction[index],
                                               f1_score(y_train_ohe, train_prediction_ohe, average='weighted'),
                                               roc_auc_score(y_train_ohe, train_prediction_ohe)]
print(f'result: {f1=:0.3f}, {auc=:0.3f}')

# RF 추론
print('[RF 검증 데이터]')
test_prediction = rf_model.predict(X_test)
print(f'{y_test=}')
print(f'{test_prediction=}')

test_prediction_ohe = ohe.transform(le.inverse_transform(test_prediction.astype(int)).reshape(-1, 1))
f1 = f1_score(y_test_ohe, test_prediction_ohe, average='weighted')
auc = roc_auc_score(y_test_ohe, test_prediction_ohe)
for index in range(test_prediction.shape[0]):
    result_table.loc[result_table.shape[0]] = [X_test_index[index, 0], 'test', 'RF', y_test_le[index, 0], test_prediction[index],
                                               f1_score(y_test_ohe, test_prediction_ohe, average='weighted'),
                                               roc_auc_score(y_test_ohe, test_prediction_ohe)]
print(f'result: {f1=:0.3f}, {auc=:0.3f}')

/home/hooni/anaconda3/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Elapsed time for RF optimization: 0.31 min
최적의 하이퍼파라미터: OrderedDict([('max_depth', 21), ('max_features', 'sqrt'), ('min_samples_leaf', 0.1), ('min_samples_split', 0.19804583485233268), ('n_estimators', 187)])
Elapsed time for RF training: 0.00 min
[RF 학습 데이터]
y_train=array([[12.,  0.,  0., ...,  0.,  1.,  0.],
       [13.,  0.,  0., ...,  0.,  0.,  1.],
       [12.,  0.,  0., ...,  0.,  1.,  0.],
       ...,
       [ 3.,  0.,  0., ...,  0.,  0.,  0.],
       [ 3.,  0.,  0., ...,  0.,  0.,  0.],
       [ 8.,  0.,  0., ...,  0.,  0.,  0.]])
train_prediction=array([ 9.64803312,  8.66438094,  8.46958942,  9.29035248,  7.67361202,
        8.30604417,  8.48259809,  8.60132631,  8.68804204,  9.54287148,
        8.22123523,  7.5309803 ,  9.1133438 ,  9.55847697,  8.91844209,
       10.16325746, 10.44992017, 10.27705163,  7.97856143,  8.66655063,
        8.9751532 , 10.66438552,  8.58084934,  9.00479518,  9.8968767 ,
        8.21594567,  8.94441676,  9.20035544,  9.39681937,  9.43465584,
      

##### 4-4. xgb

In [124]:
param_space = {
    'learning_rate': (0.01, 1.0, 'log-uniform'),
    'max_depth': (1, 50),
    'n_estimators': (50, 200),
    'min_child_weight': (1, 10),
    'subsample': (0.1, 1.0, 'uniform'),
    'gamma': (0.1, 5.0),
    'colsample_bytree': (0.1, 1.0, 'uniform'),
    'alpha': (0.0, 1.0, 'uniform'),
    'reg_lambda': (0.0, 1.0, 'uniform')
}

# 하이퍼 파라미터 튜닝
xgb_model = xgb.XGBClassifier(objective='multi:softmax')
opt = BayesSearchCV(xgb_model, param_space, n_iter=50, cv=10, n_jobs=-1, n_points=8, verbose=0)

timer = time.time()
opt.fit(X_train, y_train_le)
print(f'Elapsed time for xgb optimization: {(time.time() - timer)/60:0.2f} min')
print("최적의 하이퍼파라미터:", opt.best_params_)

# XGB 학습
timer = time.time()
xgb_model = xgb.XGBClassifier(objective='multi:softmax', **opt.best_params_)
xgb_model.fit(X_train, y_train_le)
print(f'Elapsed time for xgb training: {(time.time() - timer)/60:0.2f} min')

# label-encoding -> one-hot-encoding (결과 도출을 위한 형태)
print('[XGB 학습 데이터]')
train_prediction = xgb_model.predict(X_train)
print(f'{y_train=}')
print(f'{train_prediction=}')

train_prediction_ohe = ohe.transform(le.inverse_transform(train_prediction.astype(int)).reshape(-1, 1))
f1 = f1_score(y_train_ohe, train_prediction_ohe, average='weighted')
auc = roc_auc_score(y_train_ohe, train_prediction_ohe)
for index in range(train_prediction.shape[0]):
    result_table.loc[result_table.shape[0]] = [X_train_index[index, 0], 'train', 'XGB', y_train_le[index, 0], train_prediction[index],
                                               f1_score(y_train_ohe, train_prediction_ohe, average='weighted'),
                                               roc_auc_score(y_train_ohe, train_prediction_ohe)]
print(f'result: {f1=:0.3f}, {auc=:0.3f}')

# XGB 추론
print('[XGB 검증 데이터]')
test_prediction = xgb_model.predict(X_test)
print(f'{y_test=}')
print(f'{test_prediction=}')

test_prediction_ohe = ohe.transform(le.inverse_transform(test_prediction.astype(int)).reshape(-1, 1))
f1 = f1_score(y_test_ohe, test_prediction_ohe, average='weighted')
auc = roc_auc_score(y_test_ohe, test_prediction_ohe)
for index in range(test_prediction.shape[0]):
    result_table.loc[result_table.shape[0]] = [X_test_index[index, 0], 'test', 'XGB', y_test_le[index, 0], test_prediction[index],
                                               f1_score(y_test_ohe, test_prediction_ohe, average='weighted'),
                                               roc_auc_score(y_test_ohe, test_prediction_ohe)]
print(f'result: {f1=:0.3f}, {auc=:0.3f}')

/home/hooni/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  warnings.warn(
/home/hooni/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  warnings.warn(
/home/hooni/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  warnings.warn(
/home/hooni/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  warnings.warn(
/home/hooni/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  

Elapsed time for xgb optimization: 4.08 min
최적의 하이퍼파라미터: OrderedDict([('alpha', 0.0), ('colsample_bytree', 1.0), ('gamma', 0.1), ('learning_rate', 0.12972023227209492), ('max_depth', 29), ('min_child_weight', 10), ('n_estimators', 187), ('reg_lambda', 1.0), ('subsample', 0.7186478219085767)])
Elapsed time for xgb training: 0.05 min
[XGB 학습 데이터]
y_train=array([[12.,  0.,  0., ...,  0.,  1.,  0.],
       [13.,  0.,  0., ...,  0.,  0.,  1.],
       [12.,  0.,  0., ...,  0.,  1.,  0.],
       ...,
       [ 3.,  0.,  0., ...,  0.,  0.,  0.],
       [ 3.,  0.,  0., ...,  0.,  0.,  0.],
       [ 8.,  0.,  0., ...,  0.,  0.,  0.]])
train_prediction=array([12, 13, 12, 13,  6,  5, 13,  8,  9, 13,  5,  3,  8,  9, 13, 13, 13,
       13,  9,  3, 13, 13,  9, 13, 13,  3,  5,  9,  9,  9,  5, 13,  3,  9,
        7, 13, 13, 10, 13, 13, 13,  2,  3,  2,  3, 13,  9,  6, 13, 13,  6,
        9, 10,  8,  2,  4,  3,  8,  7,  9,  7,  8,  9, 13, 13,  5,  3,  8,
       13,  2,  5, 13,  7,  3, 13,  7, 13,  2, 12, 

In [125]:
result_table.to_excel('result/result_table_w2v.xlsx', index=False)

pd.pivot_table(result_table,
               values=['f1', 'auc'],
               index=['type', 'model'],
               aggfunc=['mean', 'std'])

mean            std     
                  auc        f1  auc   f1
type  model                              
test  LR     0.514246  0.345305  0.0  0.0
      RF     0.500885  0.031287  0.0  0.0
      SVM    0.651323  0.563953  0.0  0.0
      XGB    0.576682  0.488341  0.0  0.0
train LR     0.534068  0.418852  0.0  0.0
      RF     0.513758  0.030687  0.0  0.0
      SVM    1.000000  1.000000  0.0  0.0
      XGB    0.861077  0.937433  0.0  0.0